Город

Зарплата

Компания

Опыт


16 - Беларусь

28 - Грузия

40 - Казахстан

48 - Кыргызстан

97 - Узбекистан

113 - Россия


In [24]:
import requests

client_ID = ''
client_secret = ''
token_url = 'https://hh.ru/oauth/token'
data={'grant_type': 'client_credentials',
      'client_id': client_ID,
      'client_secret': client_secret}

response = requests.post(token_url, data=data)
access_token = response.json()['access_token']

In [27]:
!pip install fake_useragent

In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

import requests
import pandas as pd
from tqdm import tqdm
import numpy as np
import datetime
import os
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Резюме

In [29]:
def parse_age(age: list):
  if len(age) == 4:
    age_num = round(int(age[0]) + int(age[2]) / 12, 1)
  else:
    if ('год' in age[1]) | ('лет' in age[1]):
      age_num = int(age[0])
    else:
      age_num = round(int(age[0]) / 12, 1)
  return age_num

In [122]:
list_of_ress = 'архитектор гап пгс отопление-вентиляция Водоснабжение-Канализация Электроснабжение эколог BIM'.split()
new_list=[]
for i in list_of_ress:
  url = f'https://rabota.by/search/resume?text={i}&logic=normal&pos=position&exp_period=all_time&exp_company_size=any&filter_exp_period=all_time&area=97&relocation=living&age_from=&age_to=&gender=unknown&salary_from=&salary_to=&currency_code=BYR&order_by=relevance&search_period=30&items_on_page=20&no_magic=true&'
  response = requests.get(url, headers={
        'User-Agent': UserAgent().chrome
      })
  content = response.content
  parsed_content = BeautifulSoup(content, 'html.parser')
  try:
    page_count = int(parsed_content.find_all("span",{"class":"pager-item-not-in-short-range"})[-1].find('span').get_text())
    for pg in range(1, page_count+1):
      url = f'https://rabota.by/search/resume?text={i}&logic=normal&pos=position&exp_period=all_time&exp_company_size=any&filter_exp_period=all_time&area=97&relocation=living&age_from=&age_to=&gender=unknown&salary_from=&salary_to=&currency_code=BYR&order_by=relevance&search_period=30&items_on_page=20&no_magic=true&page={pg}'
      response = requests.get(url, headers={
        'User-Agent': UserAgent().chrome
      })
      content = response.content
      parsed_content = BeautifulSoup(content, 'html.parser')
      for link in parsed_content.select('a[class=serp-item__title]'):
        href = link.get('href')
        new_list.append('https://rabota.by' + href)
  except:
      url = f'https://rabota.by/search/resume?text={i}&logic=normal&pos=position&exp_period=all_time&exp_company_size=any&filter_exp_period=all_time&area=97&relocation=living&age_from=&age_to=&gender=unknown&salary_from=&salary_to=&currency_code=BYR&order_by=relevance&search_period=30&items_on_page=20&no_magic=true&page'
      response = requests.get(url, headers={
        'User-Agent': UserAgent().chrome
      })
      content = response.content
      parsed_content = BeautifulSoup(content, 'html.parser')
      for link in parsed_content.select('a[class=serp-item__title]'):
        href = link.get('href')
        new_list.append('https://rabota.by' + href)


In [123]:
len(new_list)

12

In [124]:
gender = []
age = []
status = []
city = []
position = []
salary = []
type_of_emp = []
last_work = []
educ = []
curr = []
total_exp = []
lw_exp = []
for url in new_list:
  response = requests.get(url, headers={
        'User-Agent': UserAgent().chrome
      })
  content = response.content
  parsed_content = BeautifulSoup(content, 'html.parser')
  # try:
  gender.append(str(parsed_content.find('span', {'data-qa': 'resume-personal-gender'}))[39:-7])
  # except:
  #   gender.append('unk')
  try:
    age.append(str(parsed_content.find('span', {'data-qa': 'resume-personal-age'}))[42:44])
  except:
    age.append('unk')
  try:
    status.append(str(parsed_content.find('div', {'class': 'resume-job-search-status'}))[206:-71])
  except:
    status.append('unk')
  # try:
  city.append(str(parsed_content.find('span', {'data-qa': 'resume-personal-address'}))[40:-7])
  # except:
    # city.append('unk')
  position.append(str(parsed_content.find('span', {'data-qa': 'resume-block-title-position'}))[77:-7])
  try:
    type_of_emp.append(parsed_content.find('div', {'class': 'resume-block-container'}).find('p').get_text()[11:])
  except:
    type_of_emp.append('unk')
  try:
    last_work.append(parsed_content.find('div', {'class': 'bloko-text bloko-text_strong'}).get_text())
  except:
    last_work.append('unk')
  try:
    educ.append(parsed_content.find('div', {'data-qa': 'resume-block-education-name'}).find('a').get_text())
  except:
    try:
      educ.append(parsed_content.find('div', {'data-qa': 'resume-block-education-name'}).get_text())
    except:
      educ.append('unk')
  try:
    salary.append(parsed_content.find('span', {'class': 'resume-block__salary'}).get_text().replace('\u2009', '').replace('\xa0' , ' ').split()[0])
  except:
    salary.append(0)
  try:
    curr.append((parsed_content.find('span', {'class': 'resume-block__salary'}).get_text().replace('\u2009', '').replace('\xa0' , ' ').split()[1]))
  except:
    curr.append(0)
  try:
    total_exp.append(parse_age(parsed_content.find('span', {'class': 'resume-block__title-text resume-block__title-text_sub'}).get_text().replace('\xa0', ' ')[12:].split()))
  except:
    total_exp.append('unk')
  try:
    lw_exp.append(parse_age(parsed_content.find('div', {'class': 'bloko-text bloko-text_tertiary'}).find('span').get_text().replace('\xa0' , ' ').split()))
  except:
    lw_exp.append('unl')
df = pd.DataFrame({'url': new_list,'position': position, 'salary': salary, 'total_exp': total_exp, 'currency': curr, 'status': status, 'gender':gender, 'age':age, 'city':city, 'type_of_emp':type_of_emp, 'last_work': last_work,'lw_exp': lw_exp, 'educ': educ})
df.type_of_emp = df.type_of_emp.map(lambda x:x.split(', ')[0]).replace({'unk': 'полная занятость'})
df['date'] = str(datetime.date.today())

In [98]:
def convert_res(df):
  df.currency = df.currency.replace({'Br': "BYR", '₽': 'RUB', '$': 'USD', '₸' : 'KZT', 0:'BYR', "so'm": 'UZS', '€': 'EUR'})
  df.salary = df.salary.astype('int')
  unique_curs = [i for i in df.currency.unique() if isinstance(i, str) & (i !='BYR')]
  for curr in unique_curs:
    course, scale = requests.get(f'https://api.nbrb.by/exrates/rates/{curr}?parammode=2').json()['Cur_OfficialRate'], requests.get(f'https://api.nbrb.by/exrates/rates/{curr}?parammode=2').json()['Cur_Scale']
    df.salary[df.currency==curr] = df.salary[df.currency==curr] * course / scale
  df.currency= 'BYR'
  return df

In [120]:
df = convert_res(df)

<ipython-input-98-6de666c2a260>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.salary[df.currency==curr] = df.salary[df.currency==curr] * course / scale


In [112]:
def to_google_res(df:pd.DataFrame, path = ''):
  os.chdir(f'/content/gdrive/My Drive/{path}')
  df.to_csv('resumes.xlsx')
  return print('succes!')

In [121]:
to_google_res(df, 'vacancies_kz')

succes!


# Вакансии

In [ ]:
def collects_vacancies_hh(job_title: str, area=16):
  number_of_pages = 100
#number_of_ads = number_of_pages * per_page
  job_title = job_title.split()
  for job in job_title:
      data=[]
      for i in tqdm(range(number_of_pages)):
         url = 'https://api.hh.ru/vacancies'
         par = {'text': job, 'area':str(area),'per_page':'10', 'page':i, 'search_period':'30'}
         r = requests.get(url, params=par)
         e=r.json()
         data.append(e)
         vacancy_details = data[0]['items'][0].keys()
         df = pd.DataFrame(columns= list(vacancy_details))
         ind = 0
         for i in range(len(data)):
             for j in range(len(data[i]['items'])):
                 df.loc[ind] = data[i]['items'][j]
                 ind+=1
         csv_name = job+".csv"
         df.to_csv(csv_name)
  return print('/nCollection of vacancies was finished!')

In [ ]:
def preprocessing(dataframe: pd.DataFrame, position: str):
  position = position.split()
  dataframe = dataframe[dataframe.name.str.lower().str.contains(position[0].lower(), regex=False)]
  if len(position) > 1:
    for i in [1, len(position) - 1]:
      dataframe = pd.concat([dataframe,  dataframe[dataframe.name.str.lower().str.contains(position[i].lower(), regex=False)]], axis=0)
  dataframe = dataframe[['name', 'area', 'salary', 'address', 'employer', 'schedule',  'experience', 'employment', 'alternate_url']].reset_index().drop(columns=['index'])
  df_help = dataframe[['salary', 'address']]
  for col in ['area', 'employer', 'schedule',  'experience', 'employment']:
    dataframe[col] = dataframe[col].apply(lambda x: eval(x.replace("'", '"'))).map(lambda x: x['name'])
  df_help.fillna(0, inplace=True)
  df_help['from_s'] = df_help['salary'].apply(lambda x: eval(x.replace("'", '"'))['from'] if x!=0 else 0)
  df_help['to_s'] = df_help['salary'].apply(lambda x: eval(x.replace("'", '"'))['to'] if x!=0 else 0)
  df_help['cur_s'] = df_help['salary'].apply(lambda x: eval(x.replace("'", '"'))['currency'] if x!=0 else 0)
  dataframe_final = pd.concat([dataframe, df_help[['from_s', 'to_s',	'cur_s']]], axis=1).fillna(0)
  dataframe_final = dataframe_final[dataframe_final.employer != 'ЭНЭКА'].reset_index().drop(columns=['index', 'salary', 'address'])
  dataframe_final['date'] = str(datetime.date.today())
  return dataframe_final

In [ ]:
def convert(dataframe: pd.DataFrame):
  dataframe.cur_s = dataframe.cur_s.replace('RUR', 'RUB')
  unique_curs = [i for i in dataframe.cur_s.unique() if isinstance(i, str) & (i !='BYR')]
  for curs in unique_curs:
      course, scale = requests.get(f'https://api.nbrb.by/exrates/rates/{curs}?parammode=2').json()['Cur_OfficialRate'], requests.get(f'https://api.nbrb.by/exrates/rates/{curs}?parammode=2').json()['Cur_Scale']
      dataframe.to_s[dataframe.cur_s==curs] = dataframe.to_s[dataframe.cur_s==curs] * course / scale
      dataframe.from_s[dataframe.cur_s==curs] = dataframe.from_s[dataframe.cur_s==curs] * course / scale
      dataframe.cur_s[dataframe.cur_s==curs] = 'BYR'
  dataframe.from_s=dataframe.from_s.astype('float32')
  dataframe.to_s=dataframe.to_s.astype('float32')
  return dataframe

In [ ]:
def convert_to_excel(list_of_dfs: str, excel_name: str):
  list_of_dfs = list_of_dfs.split()
  writer = pd.ExcelWriter(f'{excel_name}.xlsx')
  agg = pd.DataFrame()
  for i in list_of_dfs:
    time = eval(i)
    time.to_excel(writer, sheet_name=i)
    agg = pd.concat([agg, time], axis=0)
  agg.to_excel(writer, sheet_name='agg')
  writer.save()
  return print('Saving was finished succesfully!')

In [ ]:
def to_google_update(list_of_dfs: str):


SyntaxError: ignored

In [ ]:
def to_google_new(list_of_dfs: str):
  list_of_dfs = list_of_dfs.split()
  # os.mkdir('/content/gdrive/My Drive/vacancies_rf')
  os.chdir('/content/gdrive/My Drive/vacancies_rf')
  agg = pd.DataFrame()
  for i in list_of_dfs:
    time = eval(i)
    time.to_excel(f'{i}.xlsx')
    agg = pd.concat([agg, time], axis=0)
  agg.alternate_url.drop_duplicates(inplace=True)
  agg.to_csv('agg.xlsx')
  return print('succes!')

In [ ]:
'архитектор пгс отопление-вентиляция Водоснабжение-Канализация Электроснабжение'

In [ ]:
collects_vacancies_hh('эколог BIM', area=113)

In [ ]:
arhitektor = convert(preprocessing(pd.read_csv('/content/архитектор.csv'), 'архитектор architect'))
pgs = convert(preprocessing(pd.read_csv('/content/пгс.csv'), 'конструктор пгс'))
ov = convert(preprocessing(pd.read_csv('/content/отопление-вентиляция.csv'), 'отопление проектировщик вентиляция ОВ ок овик'))
vk = convert(preprocessing(pd.read_csv('/content/Водоснабжение-Канализация.csv'), 'проектировщик водоснабжение канализация ОВ ок овик'))
es = convert(preprocessing(pd.read_csv('/content/Электроснабжение.csv'), 'проектировщик электроснабжение э'))
ie = convert(preprocessing(pd.read_csv('/content/эколог.csv'), 'эколог'))
bim = convert(preprocessing(pd.read_csv('/content/BIM.csv'), 'bim'))

In [ ]:
to_google_new('arhitektor pgs ov vk es ie bim')

succes!
